In [20]:
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
df1 = pd.read_csv('S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')

In [22]:
df1.head(20)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4
5,Abraham_Lincoln,Did his mother die of pneumonia?,No.,easy,easy,S08_set3_a4
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months,medium,easy,S08_set3_a4
7,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months.,medium,medium,S08_set3_a4
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832,medium,easy,S08_set3_a4
9,Abraham_Lincoln,When did Lincoln begin his political career?,1832.,medium,medium,S08_set3_a4


In [23]:
all_data = df1.append([df2, df3])
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998 entries, 0 to 1457
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              3998 non-null   object
 1   Question                  3961 non-null   object
 2   Answer                    3422 non-null   object
 3   DifficultyFromQuestioner  3043 non-null   object
 4   DifficultyFromAnswerer    3418 non-null   object
 5   ArticleFile               3996 non-null   object
dtypes: object(6)
memory usage: 218.6+ KB


In [24]:
all_data['Question'] = all_data['ArticleTitle'].str.replace('_', ' ') + ' ' + all_data['Question']
all_data = all_data[['Question', 'Answer']]
all_data.shape

(3998, 2)

In [25]:
all_data.head(10)

,Question,Answer
0,Abraham Lincoln Was Abraham Lincoln the sixtee...,yes
1,Abraham Lincoln Was Abraham Lincoln the sixtee...,Yes.
2,Abraham Lincoln Did Lincoln sign the National ...,yes
3,Abraham Lincoln Did Lincoln sign the National ...,Yes.
4,Abraham Lincoln Did his mother die of pneumonia?,no
5,Abraham Lincoln Did his mother die of pneumonia?,No.
6,Abraham Lincoln How many long was Lincoln's fo...,18 months
7,Abraham Lincoln How many long was Lincoln's fo...,18 months.
8,Abraham Lincoln When did Lincoln begin his pol...,1832
9,Abraham Lincoln When did Lincoln begin his pol...,1832.


In [26]:
all_data = all_data.drop_duplicates(subset='Question')
all_data.head(10)

,Question,Answer
0,Abraham Lincoln Was Abraham Lincoln the sixtee...,yes
2,Abraham Lincoln Did Lincoln sign the National ...,yes
4,Abraham Lincoln Did his mother die of pneumonia?,no
6,Abraham Lincoln How many long was Lincoln's fo...,18 months
8,Abraham Lincoln When did Lincoln begin his pol...,1832
10,Abraham Lincoln What did The Legal Tender Act ...,"the United States Note, the first paper curren..."
12,Abraham Lincoln Who suggested Lincoln grow a b...,11-year-old Grace Bedell
14,Abraham Lincoln When did the Gettysburg addres...,1776
16,Abraham Lincoln Did Lincoln beat John C. Breck...,yes
18,Abraham Lincoln Was Abraham Lincoln the first ...,No


In [27]:
all_data.shape

(2461, 2)

In [28]:
all_data = all_data.dropna() ##Remove rows with missing values
all_data.shape

(2188, 2)

In [29]:
stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

In [30]:
def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [31]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
#The tuple represents: (document_id, token_id)
#The value following the tuple represents the tf-idf score of a given token in a given document
tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(all_data['Question']))
print(tfidf_matrix.shape)

(2188, 3550)


In [32]:
def ask_question(question):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    max_similarity = np.argmax(similarity, axis=None)
    print('Your question:', question)
    print('Closest question found:', all_data.iloc[max_similarity]['Question'])
    print('Similarity: {:.2%}'.format(similarity[0, max_similarity]))
    print('Answer:', all_data.iloc[max_similarity]['Answer'])


In [14]:
ask_question('When Abraham Lincoln started his political career')

Your question: When Abraham Lincoln started his political career
Closest question found: Abraham Lincoln Did Lincoln start his political career in 1832?
Similarity: 88.14%
Answer: Yes


In [15]:
ask_question('Where was Nicola Tesla born')

Your question: Where was Nicola Tesla born
Closest question found: Abraham Lincoln Which county was Lincoln born in?
Similarity: 40.56%
Answer: Hardin County


In [16]:
ask_question('Can whales fly')

Your question: Can whales fly
Closest question found: Otter Do sea otters have a layer of fat like whales?
Similarity: 32.08%
Answer: No


In [17]:
ask_question('Who was the third president of the United States') #Thomas Jafferson was 3rd president

Your question: Who was the third president of the United States
Closest question found: James Monroe Was James Monroe President of the United States?
Similarity: 45.33%
Answer: yes


In [18]:
ask_question('How high are crime rates in Brazil')

Your question: How high are crime rates in Brazil
Closest question found: Saint Petersburg When did the crime level become higher?
Similarity: 51.49%
Answer: After the October revolution.


In [19]:
ask_question('Which election did Grover Cleveland win')

Your question: Which election did Grover Cleveland win
Closest question found: Grover Cleveland Which election did Grover Cleveland win?
Similarity: 94.48%
Answer: 1884 and 1892 presidential elections
